# Rank Classification using BERT on Amazon Review

## Introduction

In this tutorial, you learn how to use a pre-trained Tensorflow model to classifiy a Amazon Review rank. The model was refined on Amazon Review dataset with a pretrained DistilBert model.

### About the dataset and model

[Amazon Customer Review dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html) consists of all different valid reviews from amazon.com. We will use the "Digital_software" category that consists of 102k valid reviews. As for the pre-trained model, use the DistilBERT[[1]](https://arxiv.org/abs/1910.01108) model. It's a light-weight BERT model already trained on [Wikipedia text corpora](https://en.wikipedia.org/wiki/List_of_text_corpora), a much larger dataset consisting of over millions text. The DistilBERT served as a base layer and we will add some more classification layers to output as rankings (1 - 5).

<img src="https://djl-ai.s3.amazonaws.com/resources/images/amazon_review.png" width="500">
<center>Amazon Review example</center>


## Pre-requisites
This tutorial assumes you have the following knowledge. Follow the READMEs and tutorials if you are not familiar with:
1. How to setup and run [Java Kernel in Jupyter Notebook](https://github.com/deepjavalibrary/djl/blob/master/jupyter/README.md)
2. Basic components of Deep Java Library, and how to [train your first model](https://github.com/deepjavalibrary/djl/blob/master/jupyter/tutorial/02_train_your_first_model.ipynb).


## Getting started
Load the Deep Java Libarary and its dependencies from Maven. In here, you can choose between MXNet or PyTorch. MXNet is enabled by default. You can uncomment PyTorch dependencies and comment MXNet ones to switch to PyTorch.

In [ ]:
// %mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.12.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
        
%maven ai.djl.tensorflow:tensorflow-engine:0.12.0
%maven ai.djl.tensorflow:tensorflow-api:0.12.0
%maven org.bytedeco:javacpp:1.5.4
%maven ai.djl.tensorflow:tensorflow-native-auto:2.4.1

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>com.google.protobuf</groupId>
    <artifactId>protobuf-java</artifactId>
    <version>3.8.0</version>
</dependency>

Now let's import the necessary modules:

In [ ]:
import ai.djl.*;
import ai.djl.engine.*;
import ai.djl.inference.*;
import ai.djl.modality.*;
import ai.djl.modality.nlp.*;
import ai.djl.modality.nlp.bert.*;
import ai.djl.ndarray.*;
import ai.djl.repository.zoo.*;
import ai.djl.translate.*;
import ai.djl.training.util.*;
import ai.djl.util.*;

import java.io.*;
import java.nio.file.*;
import java.util.*;

## Prepare your model files

You can download pre-trained Tensorflow model from: https://resources.djl.ai/demo/tensorflow/amazon_review_rank_classification.zip.

In [ ]:
String modelUrl = "https://resources.djl.ai/demo/tensorflow/amazon_review_rank_classification.zip";
DownloadUtils.download(modelUrl, "build/amazon_review_rank_classification.zip", new ProgressBar());
Path zipFile = Paths.get("build/amazon_review_rank_classification.zip");

Path modelDir = Paths.get("build/saved_model");
if (Files.notExists(modelDir)) {
    ZipUtils.unzip(Files.newInputStream(zipFile), modelDir);    
}

## Create Translator

Inference in deep learning is the process of predicting the output for a given input based on a pre-defined model.
DJL abstracts away the whole process for ease of use. It can load the model, perform inference on the input, and provide output.

The `Translator` interface is used to: Pre-processing and Post-processing. The pre-processing
component converts the user-defined input objects into an NDList, so that the `Predictor` in DJL can understand the
input and make its prediction. Similarly, the post-processing block receives an NDList as the output from the
`Predictor`. The post-processing block allows you to convert the output from the `Predictor` to the desired output
format.

### Pre-processing

Now, you need to convert the sentences into tokens. We provide a powerful tool `BertTokenizer` that you can use to convert questions and answers into tokens, and batchify your sequence together. Once you have properly formatted tokens, you can use `Vocabulary` to map your token to BERT index.

The following code block demonstrates tokenizing the question and answer defined earlier into BERT-formatted tokens.

In the zip file, we also bundled the BERT `vocab.txt` file.

In [ ]:
// Prepare the vocabulary
Path vocabFile = modelDir.resolve("vocab.txt");
SimpleVocabulary vocabulary = SimpleVocabulary.builder()
        .optMinFrequency(1)
        .addFromTextFile(vocabFile)
        .optUnknownToken("[UNK]")
        .build();
BertFullTokenizer tokenizer = new BertFullTokenizer(vocabulary, true);
int maxTokenLength = 64; // cutoff tokens length


In [ ]:
class MyTranslator implements Translator<String, Classifications> {

    private BertFullTokenizer tokenizer;
    private SimpleVocabulary vocab;
    private List<String> ranks;
    private int length;

    public MyTranslator(BertFullTokenizer tokenizer, int length) {
        this.tokenizer = tokenizer;
        this.length = length;
        vocab = tokenizer.getVocabulary();
        ranks = Arrays.asList("1", "2", "3", "4", "5");
    }

    @Override
    public Batchifier getBatchifier() {
        return new StackBatchifier();
    }

    @Override
    public NDList processInput(TranslatorContext ctx, String input) {
        List<String> tokens = tokenizer.tokenize(input);
        long[] indices = new long[length];
        long[] mask = new long[length];
        long[] segmentIds = new long[length];
        int size = Math.min(length, tokens.size());
        for (int i = 0; i < size; i++) {
            indices[i + 1] = vocab.getIndex(tokens.get(i));
        }
        Arrays.fill(mask,  0, size, 1);
        NDManager m = ctx.getNDManager();
        return new NDList(m.create(indices), m.create(mask), m.create(segmentIds));
    }

    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        return new Classifications(ranks, list.singletonOrThrow().softmax(0));
    }
}


## Load your model

In [ ]:
MyTranslator translator = new MyTranslator(tokenizer, maxTokenLength);

Criteria<String, Classifications> criteria = Criteria.builder()
        .setTypes(String.class, Classifications.class)
        .optModelPath(modelDir) // Load model form model directory
        .optTranslator(translator) // use custom translaotr 
        .build();

ZooModel<String, Classifications> model = criteria.loadModel();

## Run inference

Lastly, we will need to create a predictor using our model and translator. Once we have a predictor, we simply need to call the predict method on our test image.

In [ ]:
String review = "It works great, but it takes too long to update itself and slows the system";

Predictor<String, Classifications> predictor = model.newPredictor();
Classifications classifications = predictor.predict(review);

classifications